A preprocessing notebook that prepares the data for upload to milvus.

In [ ]:
from tqdm import tqdm
import re 

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, DirectoryLoader
from src.utils import reformat_text
from src.vectorstore import jsonize_document

import pandas as pd
import json

In [ ]:
loader = DirectoryLoader("docs/youtube/", glob="*.txt", show_progress=True)
docs = loader.load()

In [ ]:
import os

def list_txt_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            print(filename)

# Example usage:
list_txt_files('docs/youtube/')


In [ ]:
for doc in docs:
    print(doc.metadata)

In [ ]:
for i, doc in enumerate(docs):
    doc.page_content = reformat_text(doc.page_content)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100,
    length_function=len,
)

In [ ]:
docs

In [ ]:
splitted_docs = text_splitter.split_documents(docs)

In [ ]:
len(splitted_docs)

In [ ]:
jsons = []

for d in tqdm(splitted_docs, desc="Processing texts"):
    json = jsonize_document(d)
    jsons.append(json)

In [ ]:
import json
with open('jsons.json', 'w') as file:
    json.dump(jsons, file, indent=4)

In [ ]:
df = pd.read_json("/home/damir/Projects/huberman_rag/jsons.json")
df['pk'] = df.index
df.to_csv("dataframe.csv", index=False)